In [2]:
using CSV, DataFrames, GMT
using Graphs, GraphIO, FileIO
using Geodesy

include("./src/cubes.jl")
include("./src/network.jl")

add_properties (generic function with 1 method)

In [ ]:
function network_plot_gmt_Vrancea(region::String, cell_size::Float64, minimum_magnitude::Int64, marker_size::Float64, ratio_for_z::Int64, legend_info::Vector{Any}, perspective::Tuple{Int64,Int64}, quality::Int64, colorbar_info::Vector{Any})
    """
    Description:
    Takes in the region, cell_size and minimum_magnitude and plots with GMT
        a 3D visualization of the earthquake network with the above specifications, 
        with a semi-3D relief map of the region.

    Parameters:
    - `region`: a string (capital letter) the seismic region under scrutiny.
    - `cell_size`: discretization length of the cubes being split into.
    - `minimum_magnitude`: minimum magnitude to filter the database
    - `persepctive`: tuple (x,y) to set the view of the 3D plot
    - `quality`: the quality of the relief map, higher is better (>1000), and computationally demanding

    Returns:
        Plots the GMT visualization.
    """
    
    ##################################### READ DATA #####################################
    path = "./data/"
    filepath = path * "Romania" * ".csv"
    mkpath("./gmt/$region")
    df = CSV.read(filepath, DataFrame);
    
    # Read Vrancea
    # Read data
    path = "./data/"
    filepath_Vrancea = path * region * ".csv"

    df_Vrancea = CSV.read(filepath_Vrancea, DataFrame);
    minlon_Vrancea = minimum(df_Vrancea.Longitude)
    maxlon_Vrancea = maximum(df_Vrancea.Longitude)
    minlat_Vrancea = minimum(df_Vrancea.Latitude)
    maxlat_Vrancea = maximum(df_Vrancea.Latitude)

    df = df[(df.Longitude .> minlon_Vrancea) .& (df.Longitude .< maxlon_Vrancea) .& 
        (df.Latitude .> minlat_Vrancea) .& (df.Latitude .< maxlat_Vrancea), :]

    df_filtered = df[df.Magnitude .> minimum_magnitude,:] 
    #########################################################################################

    ##################################### MAP OF REGION #####################################
    #########################################################################################
    # Split into cubes
    df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);
    # Region coords based on cubes
    min_lon = minimum(df_filtered_cubes.cubeLongitude) -0.2
    max_lon = maximum(df_filtered_cubes.cubeLongitude) +0.2
    min_lat = minimum(df_filtered_cubes.cubeLatitude) -0.2
    max_lat = maximum(df_filtered_cubes.cubeLatitude) +0.2
    min_dep = minimum(df_filtered_cubes.cubeDepth);
    max_dep = maximum(df_filtered_cubes.cubeDepth);
    # Create the map coordinates
    map_coords = [min_lon,max_lon,min_lat,max_lat]
    map_coords_depth = [min_lon,max_lon,min_lat,max_lat,-max_dep,-min_dep]
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    # z_ratio (ratio between longitude and depth, exaggerated by a factor of 2 for better vis)
    x0_lla = LLA(min_lat,min_lon,-min_dep)
    xf_lla = LLA(min_lat,max_lon,-min_dep)
    lon_dist_in_km = Geodesy.euclidean_distance(xf_lla,x0_lla) / 1000
    z_ratio = ratio_for_z * max_dep / lon_dist_in_km

    #########################################################################################

    ####################################### GMT PLOT ########################################
    #########################################################################################
    #Basemap to define the axes, annot and labels
    basemap(limits=map_coords_depth, proj=:merc, zsize=z_ratio, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", 
            par=(FONT_LABEL="20p,Palatino-Roman",MAP_LABEL_OFFSET=0.8, FONT_ANNOT="18p,Palatino-Roman,black"), view=perspective)
    
    
    # Edges, plotted manually
    # Create network
    MG = create_network(df_filtered, df_filtered_cubes)
    # edgelist_array = Matrix(edgelist); for edges 
    edgelist = collect(edges(MG)) |> DataFrame;
    for i in range(1,nrow(edgelist))
        line_coords = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                        lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                        deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])
    
        plot3d!(line_coords.lons, line_coords.lats, -line_coords.deps, JZ=string(z_ratio) * "c", proj=:merc, pen=(0.3,:black), alpha=65, view=perspective)
    end
    
    # Nodes
    # connectivity for nodes degree
    connectivity = degree(MG);
    # C_markers = makecpt(cmap=:berlin, range=(minimum(connectivity),maximum(connectivity)), inverse=true);
    # C_markers = makecpt(cmap=:berlin, range=(minimum(connectivity),maximum(connectivity)));
    C_markers = makecpt(cmap=:split, range=(minimum(connectivity),maximum(connectivity)));

    scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
                limits=map_coords_depth,frame="SEnwZ1+b",proj=:merc, marker=:cube, markersize=marker_size,
                cmap=C_markers, zcolor=connectivity, alpha=60, view=perspective)
    
    # Colorbar
    colorbar!(limits=map_coords, pos=(paper=colorbar_info[1], size=colorbar_info[2]), shade=0.4, xaxis=(annot=colorbar_info[3]), frame=(xlabel="Degree",),
                par=(FONT_LABEL="22p,Palatino-Roman,black",MAP_LABEL_OFFSET=0.6,FONT_ANNOT="18p,Palatino-Roman,black"),view=(180,90))
    
    # Relief map
    grdview!(relief_map, proj=:merc, surftype=(image=quality,), 
                cmap=C_map, zsize=1.0, alpha=25 ,yshift=z_ratio-0.6, view=perspective,
                # savefig="./gmt/$region/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png",
                )


    legend!((label=(txt="L=$(cell_size) , M"*subscript(min)*"=$(minimum_magnitude) ", justify=:L, font=(20, "Palatino-Roman")),),
                limits=map_coords, pos=(paper=legend_info[1], width=legend_info[2], justify=:BL, spacing=2.4),
                clearance=(0.1,0.1), box=(pen=0.1, fill=:azure1),
                figsize=10, proj=:Mercator, view = (180,90),
                savefig="./gmt/$region/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png",)
    #########################################################################################
end
#############################################################################################################################################

In [ ]:
# quality = 1500
# region = "Vrancea"
# ratio_for_z = 8
# perspective = (155,25)
# for cell_size in [5.0, 10.0]
#     if cell_size == 5.0
#         marker_size = 0.4
#         legend_info = [(-2.5,8.0),4.9]
#     elseif cell_size == 10.0
#         marker_size = 0.55
#         legend_info = [(-2.5,8.0),5.1]
#     end

#     for minimum_magnitude in [4,3,2]
#         if minimum_magnitude == 4
#             colorbar_info = [(22.0,8.5),(12.0,0.5),2]
#         elseif minimum_magnitude == 3
#             colorbar_info = [(22.0,8.5),(12.0,0.5),10]

#         elseif minimum_magnitude == 2
#             colorbar_info = [(22.0,8.5),(12.0,0.5),14]

#         end
#         network_plot_gmt_Vrancea(region, cell_size, minimum_magnitude, marker_size, ratio_for_z, legend_info, perspective, quality,  colorbar_info)

#     end

# end

In [99]:
quality = 100
region = "Vrancea"
ratio_for_z = 8
perspective = (155,25)
cell_size = 10.0
marker_size = 1.2
minimum_magnitude = 2
legend_info = [(-2.5,8.0),5.1]
colorbar_info = [(22.0,8.5),(12.0,0.5),14]

3-element Vector{Any}:
   (22.0, 8.5)
   (12.0, 0.5)
 14

In [100]:
iterations = 350

350

In [101]:
path = "./data/"
filepath = path * "Romania" * ".csv"
mkpath("./gmt/$region")
df = CSV.read(filepath, DataFrame);

# Read Vrancea
# Read data
path = "./data/"
filepath_Vrancea = path * region * ".csv"

df_Vrancea = CSV.read(filepath_Vrancea, DataFrame);
minlon_Vrancea = minimum(df_Vrancea.Longitude)
maxlon_Vrancea = maximum(df_Vrancea.Longitude)
minlat_Vrancea = minimum(df_Vrancea.Latitude)
maxlat_Vrancea = maximum(df_Vrancea.Latitude)

df = df[(df.Longitude .> minlon_Vrancea) .& (df.Longitude .< maxlon_Vrancea) .& 
    (df.Latitude .> minlat_Vrancea) .& (df.Latitude .< maxlat_Vrancea), :]

df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

#########################################################################################

##################################### MAP OF REGION #####################################
#########################################################################################
# Split into cubes
df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);
# Region coords based on cubes
min_lon = minimum(df_filtered_cubes.cubeLongitude) -0.2
max_lon = maximum(df_filtered_cubes.cubeLongitude) +0.2
min_lat = minimum(df_filtered_cubes.cubeLatitude) -0.2
max_lat = maximum(df_filtered_cubes.cubeLatitude) +0.2
min_dep = minimum(df_filtered_cubes.cubeDepth);
max_dep = maximum(df_filtered_cubes.cubeDepth);
# Create the map coordinates
map_coords = [min_lon,max_lon,min_lat,max_lat]
map_coords_depth = [min_lon,max_lon,min_lat,max_lat,-max_dep,-min_dep]
# Colormap for the region topography
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
# Relief map of the region
relief_map = grdcut("@earth_relief_30s", region=map_coords);

# z_ratio (ratio between longitude and depth, exaggerated by a factor of 2 for better vis)
x0_lla = LLA(min_lat,min_lon,-min_dep)
xf_lla = LLA(min_lat,max_lon,-min_dep)
lon_dist_in_km = Geodesy.euclidean_distance(xf_lla,x0_lla) / 1000
z_ratio = ratio_for_z * max_dep / lon_dist_in_km

#########################################################################################

####################################### GMT PLOT ########################################
#########################################################################################
#Basemap to define the axes, annot and labels
basemap(limits=map_coords_depth, proj=:merc, zsize=z_ratio, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", 
        par=(FONT_LABEL="20p,Palatino-Roman",MAP_LABEL_OFFSET=0.8, FONT_ANNOT="18p,Palatino-Roman,black"), view=perspective)


# deleteat!(df_filtered, iterations+1:nrow(df_filtered))
# df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);
# Edges, plotted manually
# Create network
MG = create_network(df_filtered, df_filtered_cubes)
# edgelist_array = Matrix(edgelist); for edges 
edgelist = collect(edges(MG)) |> DataFrame;
for i in range(1,nrow(edgelist))
    line_coords = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line_coords.lons, line_coords.lats, -line_coords.deps, JZ=string(z_ratio) * "c", proj=:merc, pen=(1.4,:black), alpha=65, view=perspective)
end

# Nodes
# connectivity for nodes degree
connectivity = degree(MG);
# C_markers = makecpt(cmap=:berlin, range=(minimum(connectivity),maximum(connectivity)), inverse=true);
# C_markers = makecpt(cmap=:berlin, range=(minimum(connectivity),maximum(connectivity)));
C_markers = makecpt(cmap=:split, range=(minimum(connectivity),maximum(connectivity)));

scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
            limits=map_coords_depth,frame="SEnwZ1+b",proj=:merc, marker=:cube, markersize=marker_size, cmap=C_markers, #cmap=:black,
             zcolor=connectivity, alpha=60, view=perspective)

# # Colorbar
# colorbar!(limits=map_coords, pos=(paper=colorbar_info[1], size=colorbar_info[2]), shade=0.4, xaxis=(annot=colorbar_info[3]), frame=(xlabel="Degree",),
#             par=(FONT_LABEL="22p,Palatino-Roman,black",MAP_LABEL_OFFSET=0.6,FONT_ANNOT="18p,Palatino-Roman,black"),view=(180,90))

# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=quality,), 
            cmap=C_map, zsize=0.2, alpha=25 ,yshift=z_ratio-0.6, view=perspective,
            # savefig="./gmt/$region/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png",
            )


legend!((label=(txt="L=$(cell_size) , M"*subscript(min)*"=$(minimum_magnitude) ", justify=:L, font=(20, "Palatino-Roman")),),
            limits=map_coords, pos=(paper=legend_info[1], width=legend_info[2], justify=:BL, spacing=2.4),
            clearance=(0.1,0.1), box=(pen=0.1, fill=:azure1),
            figsize=10, proj=:Mercator, view = (180,90),
            savefig="./gmt/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude)_$(iterations).png",)

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 25.8666 to 25.8583333333
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 27.1577 to 27.1583333333
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 45.055 to 45.05
grdcut [WARNING]: (n - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: n reset from 46.1748 to 46.175


In [61]:
edgelist

Row,src,dst
,Int64,Int64
1,1,2


In [102]:
df_filtered

Row,Datetime,Latitude,Longitude,Depth,Magnitude,cubeIndex,energyRelease
,DateTime,Float64,Float64,Float64,Float64,String,Float64
1,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4,699,3.16228e9
2,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0,1053,794.328
3,1977-03-05T02:35:22,45.63,26.19,121.3,3.0,877,794.328
4,1977-03-05T12:08:42,45.37,26.3,124.0,3.5,883,4466.84
5,1977-03-06T13:27:06.300,45.83,26.63,112.3,2.6,834,199.526
6,1977-03-11T21:19:07.300,45.49,26.76,87.1,2.2,624,50.1187
7,1977-03-12T13:27:13.300,45.74,26.9,99.9,3.8,716,12589.3
8,1977-03-12T18:02:15.200,45.79,26.89,100.0,3.5,780,4466.84
9,1977-03-12T22:03:44.400,45.73,26.8,97.9,2.6,707,199.526


In [76]:
df_filtered_cubes

Row,cubeIndex,xLatitude,yLongitude,zDepth,cubeLatitude,cubeLongitude,cubeDepth
,String,Int64,Int64,Int64,Float64,Float64,Float64
1,6,7,1,1,45.8049,26.7137,99.0
2,28,1,1,5,45.265,26.7137,139.0


In [77]:
connectivity

2-element Vector{Int64}:
 1
 1